In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [27]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return (soup.find_all('thead'), soup.find_all('tbody'))

In [32]:
html_soup = get_soup('https://www.worldometers.info/coronavirus/')

In [39]:
def extract_content(soup, index):
    content = {}
    headers = []
    rows = []
    for th in soup[0][index].find('tr').find_all('th'):
        headers.append(th.text.strip())
    content['headers'] = headers
    for tr in soup[1][index].find_all('tr')[1:]:
        cells = []
        # grab all td tags in this table row
        tds = tr.find_all('td')
        if len(tds) == 0:
            # if no td tags, search for th tags
            ths = tr.find_all('th')
            for th in ths:
                cells.append(th.text.strip())
        else:
            # use regular td tags
            for td in tds:
                cells.append(td.text.strip())
        rows.append(cells)
    content['rows'] = rows
    return content

In [40]:
content = extract_content(html_soup, 0)

In [41]:
content['headers'][0] = 'NUMBER'
content['headers'][1] = 'COUNTRY_NAME'
content['headers'][2] = 'TOTAL_CASES'
content['headers'][3] = 'NEW_CASES'
content['headers'][4] = 'TOTAL_DEATHS'
content['headers'][5] = 'NEW_DEATHS'
content['headers'][6] = 'TOTAL_RECOVERED'
content['headers'][7] = 'NEW_RECOVERED'
content['headers'][8] = 'ACTIVE_CASES'
content['headers'][9] = 'CRITICAL_CASES'
content['headers'][10] = 'TOTAL_CASES_1M_POP'
content['headers'][11] = 'DEATHS_1M_POP'
content['headers'][12] = 'TOTAL_TESTS'
content['headers'][13] = 'TESTS_1M_POP'
content['headers'][14] = 'POPULATION'
content['headers'][15] = 'CONTINENT'
content['headers'][16] = '1_CASE_EVERY_X_PPL'
content['headers'][17] = '1_DEATH_EVERY_X_PPL'
content['headers'][18] = '1_TEST_EVERY_X_PPL'

In [46]:
df = pd.DataFrame(content['rows'], columns=content['headers'])
df_subset = df[['COUNTRY_NAME', 'TOTAL_CASES', 'TOTAL_DEATHS', 'POPULATION']]
df_subset

,COUNTRY_NAME,TOTAL_CASES,TOTAL_DEATHS,POPULATION
0,South America,"1,146,755","51,046",
1,Europe,"2,094,324","178,943",
2,Asia,"1,366,210","35,094",
3,Africa,"192,118","5,215",
4,Oceania,"8,875",124,
...,...,...,...,...
217,St. Barth,6,,"9,875"
218,Lesotho,4,,"2,141,162"
219,Anguilla,3,,"14,994"
220,Saint Pierre Miquelon,1,,"5,796"


In [49]:
df_subset.loc[df_subset['COUNTRY_NAME'] == 'Brazil']

,COUNTRY_NAME,TOTAL_CASES,TOTAL_DEATHS,POPULATION
8,Brazil,"691,962","37,312","212,463,372"
